<a href="http://landlab.github.io"><img style="float: left" src="../../landlab_header.png"></a>

# Write NetCDF output of sediment parcels. 

<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>
<hr>


Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
from landlab.components import FlowDirectorSteepest, NetworkSedimentTransporter
from landlab.data_record import DataRecord
from landlab.grid.network import NetworkModelGrid
from landlab.plot import graph
from landlab.io import read_shapefile
from landlab import ExampleData
import xarray as xr
from landlab.plot import plot_network_and_parcels
%matplotlib inline

Make grid 

In [ ]:
datadir = ExampleData("io/shapefile", case="methow").base

shp_file = datadir / "MethowSubBasin.shp"
points_shapefile = datadir / "MethowSubBasin_Nodes_4.shp"

grid = read_shapefile(
    shp_file,
    points_shapefile=points_shapefile,
    node_fields=["usarea_km2", "Elev_m"],
    link_fields=["usarea_km2", "Length_m"],
    link_field_conversion={"usarea_km2": "drainage_area", "Slope":"channel_slope", "Length_m":"reach_length"},
    node_field_conversion={
        "usarea_km2": "drainage_area",
        "Elev_m": "topographic__elevation",
    },
    threshold=0.01,
    )
grid.at_node["bedrock__elevation"] = grid.at_node["topographic__elevation"].copy()

grid.at_link["channel_width"] = 1 * np.ones(grid.number_of_links) # m

grid.at_link["flow_depth"] = 0.5 * np.ones(grid.number_of_links) # m

Create parcels. 

In [ ]:
# element_id is the link on which the parcel begins. 
element_id = np.repeat(np.arange(grid.number_of_links), 10)
element_id = np.expand_dims(element_id, axis=1)

volume = 1*np.ones(np.shape(element_id))  # (m3)
active_layer = np.ones(np.shape(element_id)) # 1= active, 0 = inactive
density = 2650 * np.ones(np.size(element_id))  # (kg/m3)
abrasion_rate = 0 * np.ones(np.size(element_id)) # (mass loss /m)

# Lognormal GSD
medianD = 0.15 # m
mu = np.log(medianD)
sigma = np.log(2) #assume that D84 = sigma*D50
np.random.seed(0)
D = np.random.lognormal(
    mu,
    sigma,
    np.shape(element_id)
)  # (m) the diameter of grains in each parcel
time_arrival_in_link = np.random.rand(np.size(element_id), 1) 
location_in_link = np.random.rand(np.size(element_id), 1) 
lithology = ["quartzite"] * np.size(element_id)
variables = {
    "abrasion_rate": (["item_id"], abrasion_rate),
    "density": (["item_id"], density),
    "lithology": (["item_id"], lithology),
    "time_arrival_in_link": (["item_id", "time"], time_arrival_in_link),
    "active_layer": (["item_id", "time"], active_layer),
    "location_in_link": (["item_id", "time"], location_in_link),
    "D": (["item_id", "time"], D),
    "volume": (["item_id", "time"], volume)
}

items = {"grid_element": "link", "element_id": element_id}

parcels = DataRecord(
    grid,
    items=items,
    time=[0.0],
    data_vars=variables,
    dummy_elements={"link": [NetworkSedimentTransporter.OUT_OF_NETWORK]},
)

Set up time

In [ ]:
timesteps = 15 # total number of timesteps
pulse_timestep = 7
pulse_link = 27
dt = 60 * 60 * 24 *2 # length of timestep (seconds) 

Create pulse parcels, put them all on link 27:

In [ ]:

num_pulse_parcels = 100

newpar_element_id = np.zeros(num_pulse_parcels, dtype=int) + pulse_link
newpar_element_id = np.expand_dims(newpar_element_id, axis=1)

new_starting_link = np.squeeze(newpar_element_id)

np.random.seed(0)

new_time_arrival_in_link = pulse_timestep * dt * np.ones(np.shape(newpar_element_id))

new_volume = 0.5 * np.ones(
    np.shape(newpar_element_id)
)  # (m3) the volume of each parcel

new_lithology = ["pulse_material"] * np.size(
    newpar_element_id
)  # a lithology descriptor for each parcel

new_active_layer = np.ones(
    np.shape(newpar_element_id)
)  # 1 = active/surface layer; 0 = subsurface layer

new_density = 2650 * np.ones(np.size(newpar_element_id))  # (kg/m3)

new_location_in_link = np.random.rand(np.size(newpar_element_id), 1)

new_abrasion_rate = 0 * np.ones(np.size(newpar_element_id))

new_D = 0.1 * np.ones(np.shape(newpar_element_id))

# smaller lognormal GSD
medianD = 0.1 # m
mu = np.log(medianD)
sigma = np.log(2) #assume that D84 = sigma*D50
np.random.seed(0)
new_D = np.random.lognormal(
    mu,
    sigma,
    np.shape(newpar_element_id)
)  # (m) the diameter of grains in each parcel

newpar_grid_elements = np.array(
    np.empty((np.shape(newpar_element_id)), dtype=object)
)  # BUG: should be able to pass ["link"], but datarecord fills it into an incorrect array shape-- the length of parcels (NOT new parcels)
newpar_grid_elements.fill("link")

new_parcels = {
    "grid_element": newpar_grid_elements,
    "element_id": newpar_element_id,
}

new_variables = {
    "starting_link": (["item_id"], new_starting_link),
    "abrasion_rate": (["item_id"], new_abrasion_rate),
    "density": (["item_id"], new_density),
    "lithology": (["item_id"], new_lithology),
    "time_arrival_in_link": (["item_id", "time"], new_time_arrival_in_link),
    "active_layer": (["item_id", "time"], new_active_layer),
    "location_in_link": (["item_id", "time"], new_location_in_link),
    "D": (["item_id", "time"], new_D),
    "volume": (["item_id", "time"], new_volume),
}




Before running the NST, we need to determine flow direction on the grid (upstream and downstream for each link). To do so, we initalize and run a Landlab flow director component: 

In [ ]:
fd = FlowDirectorSteepest(grid, "topographic__elevation")
fd.run_one_step()

Then, we initialize the network sediment transporter: 

In [ ]:
nst = NetworkSedimentTransporter(    
    grid,
    parcels,
    fd,
    bed_porosity=0.3,
    g=9.81,
    fluid_density=1000,
    transport_method="WilcockCrowe",
)

Now we are ready to run the model forward in time: 

In [ ]:
files = []
for i in range(timesteps):
    if i == pulse_timestep:
        parcels.add_item(time=[nst.time], new_item=new_parcels, new_item_spec=new_variables)
        
    nst.run_one_step(dt)
    
    if i>1 and np.remainder(i, 5) == 4: 
        fn = "file_{i}.nc".format(i=i)
        files.append(fn)
        if i+1 == timesteps:
            parcels.dump_prior_timesteps_to_netcdf(fn, include_final_timestep=True)
        else:
            parcels.dump_prior_timesteps_to_netcdf(fn)

This is now tricky, b/c plotting expects a DataRecord, not an xarray dataset. Difference is parcels.dataset vs parcels_ds.

Right now I'm going to hack this by private var assignment. My suspicion is that the right thing to do would be to make it possible to make create a datarecord from grid and dataset e.g.


```python
grid = (recreate grid from files)
parcels_ds = load_from_files
parcels = DataRecord.from_grid_and_dataset(grid, parcels_ds)
```

In [ ]:
parcels_ds = xr.open_mfdataset(files)

dummy_elements={"link": [NetworkSedimentTransporter.OUT_OF_NETWORK]}

parcels = DataRecord.from_grid_and_dataset(grid, parcels_ds, dummy_elements=dummy_elements)

The plot below is an example of accessing variables associated with the grid (`grid.at_link.X`, or `grid.at_node.X`), as well as a variable associated with this instance of NetworkModelGrid (`nmg.X`):

In [ ]:
for i in range(timesteps):

    fig = plot_network_and_parcels(
        grid, parcels,
        parcel_time_index=i,
        parcel_alpha = 0.6
    )
    plt.show()

In [ ]:

import os
for file in files:
    os.remove(file)